In [21]:
from system import *
s1 = system('2',MMA=True,ManyVariables=False,TwoPhotonResonance= False)

var('gamma','DE','De','g','g_f','Omega','v','gamma_f','gamma_g','gamma0','De0','g0','db',domain='positive')
var('DEt',domain='positive',  latex_name =r'\tilde{\Delta }_E')
var('Det',domain='positive',  latex_name =r'\tilde{\Delta }_e')
var('De0t',domain='positive',  latex_name =r'\tilde{\Delta }_{e0}')
var('dct',domain='complex',  latex_name =r'\tilde{\delta }_c')
var('dbt',domain='complex',  latex_name =r'\tilde{\delta }_c')
var('DEg',domain='positive',  latex_name =r'\Delta_{E\gamma}')
var('Deg',domain='positive',  latex_name =r'\Delta_{e\gamma}')
var('kappa_c','kappa_b',domain='positive')
var('C',domain='positive')
var('c',domain='positive',  latex_name =r'c')

Initializing system 2 ...
Constructing states and excitations ...
Constructing ground and first-excited statespace ...
Constructing gs_hamiltonian ...
Constructing e1_hamiltonian ...
Constructing interactions V_plus and V_minus ...
Constructing NJ_hamiltonian ...
Inverting NJ_hamiltonian ...
Constructing eff_hamiltonian and effective lindblad operators ...

System  2  initialized in 0.9 seconds.


c

# Effective Hamiltonian as Real part

Will output $Q_n$ from expression:

$$\Delta_n = - \dfrac{\Omega^2}{\gamma} \text{Re}\{Q_n \} $$

In [11]:
diag = 0  #input n corresponding to diag element to view



#Take only one of the compl conj terms, choose the one with the shortest length when shortned
min_expr_len = 10**10
for term in range(2):
    b = s1.eff_hamiltonian_gs[diag,diag]
    #b = b.subs(gamma0=gamma)
    
    b = b.operands()[1].operands()[term]  #  + b.operands()[1].operands()[3] use if obtained with sage


    b = b.subs(DE=+0.5*I*gamma_f + 0.5*I*gamma_g + DEt*gamma)
    b = b.subs(De=+0.5*I*gamma   +Det*gamma)
    
    b = b.subs(De0= +0.5*I*gamma0 +De0t*gamma)
    
    #var('a0', latex_name =r'a_0')
    #b = b.subs(De0=+0.5*I*gamma   +Det*gamma*a0)
    #var('n')
    b = b.subs(DE=DEg*gamma)
    b = b.subs(De=Deg*gamma)
    var('a0', latex_name =r'a_0')
    b = b.subs(De0=a0*Deg*gamma)
    
    var('R_f',domain='real')#ratio  (g_f/g)^2
    b = b.subs(g_f=g*sqrt(R_f))
    
    var('R0',domain='positive',  latex_name =r'R_0')
    b = b.subs(g0=g*sqrt(R0))

    var('R_v',domain='real',  latex_name =r'R_{\nu}') #ratio (v/g)^2
    b = b.subs(v=g*sqrt(R_v))

    var('r_b',domain='real')
    b = b.subs(kappa_b=gamma/r_b)


    var('r_g',domain='real',latex_name =r'r_g')
    b = b.subs(gamma_g=gamma*r_g)


    var('r_f',domain='real',latex_name =r'r_f')
    b = b.subs(gamma_f=gamma*r_f)
    
    var('r0',domain='real',latex_name =r'r_0')
    b = b.subs(gamma0=gamma*r0)


    b = b.subs(g = sqrt(C*kappa_c*gamma)) 
    
    b = b.subs(dc = (dct )*kappa_c  )
    b = b.subs(db = (dbt )*kappa_b  )
    b = b.subs(r_b = c/(C*R_v))
    
    b = b*gamma  #Omega has to be substituted with Omega/gamma
    

    b = b._mathematica_().Factor()._sage_()
    b = MMA_simplify(b,full= False)
    #holds the smallest expression
    if len(str(b))<min_expr_len:
        min_expr_len = len(str(b))
        h_eff = b
print('Element is the real part of this expression times  Ω^2/γ')
show( (symround(h_eff,digits=1)))

Element is the real part of this expression times  Ω^2/γ


1/(-2*I*C*R_f + DEt)

### Show the result separated in Numerator and denominator multiplying by a factor

In [60]:
factor = 1#*16*4


print('numerator')
nu = symround(SR(str(MMA_simplify(factor*(h_eff.numerator().expand()),full=True)).replace('Sqrt','sqrt'))\
              ,digits=3)
show(nu)
print("\n\n Latex")
print(latex(nu))

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print('\n\ndenominator')
den = symround(SR(str(MMA_simplify(factor*(h_eff.denominator().expand()),full= not True)).replace('Sqrt','sqrt'))\
               ,digits=3)
show(den)
print("\n\n Latex")
print(latex(den))

numerator


-2*Det*c*dct - 0.500*I*Det*dct^2 + C*c + 0.500*I*C*dct



 Latex
-2 \, {\tilde{\Delta }_e} {c} {\tilde{\delta }_c} - 0.500i \, {\tilde{\Delta }_e} {\tilde{\delta }_c}^{2} + C {c} + 0.500i \, C {\tilde{\delta }_c}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


denominator


(Det*c + 0.500*I*Det*dct - 0.500*I*C)*C*R_f - (2*Det*c*dct + 0.500*I*Det*dct^2 - C*c - 0.500*I*C*dct)*DEt



 Latex
{\left({\tilde{\Delta }_e} {c} + 0.500i \, {\tilde{\Delta }_e} {\tilde{\delta }_c} - 0.500i \, C\right)} C R_{f} - {\left(2 \, {\tilde{\Delta }_e} {c} {\tilde{\delta }_c} + 0.500i \, {\tilde{\Delta }_e} {\tilde{\delta }_c}^{2} - C {c} - 0.500i \, C {\tilde{\delta }_c}\right)} {\tilde{\Delta }_E}


# Lindblad Operators

Choose which of the Lindblad operators to view and which non zero element of it

In [12]:

def lind(lind_op,which):    
    L_matrix_orig = s1.lindblad_list[lind_op]
    L_matrix      = s1.eff_lindblad_list[lind_op] #lindblad_list[lind_op] ##(gamma_g=0,gamma_f=gamma)

    L_nonzeros = []
    L_nonzeros_pos = []
    #find non zero elements
    for i in  range(L_matrix.nrows()):
        for j in  range(L_matrix.ncols()):
            if not L_matrix[i,j].is_zero():
                L_nonzeros.append(L_matrix[i,j])
                L_nonzeros_pos.append((i,j))

    #show what these elements do 
    #for t in L_nonzeros_pos:
    #    print(f' |{s1.gs_e1_dec_states[t[0]]}> < {s1.gs_e1_dec_states[t[1]]} | ')

    L_elem = L_nonzeros[which]

    var('gamma_f')
    var('R0',domain='real',latex_name =r'R_0')
    L_elem = L_elem.subs(DE=+0.5*I*gamma_f + 0.5*I*gamma_g + DEt*gamma)
    L_elem = L_elem.subs(De=+0.5*I*gamma   +Det*gamma)
    L_elem = L_elem.subs(De0=+0.5*I*gamma0   +De0t*gamma*R0)
    #L_elem = L_elem.subs(DE=gamma*DEg)
    #L_elem = L_elem.subs(De=gamma*Deg)                     
    #COUPLINGS
    b = L_elem
    #ratio  (g_0/g)^2
    b = b.subs(g0=g*sqrt(R0))
    
    var('R_f',domain='real')#ratio  (g_f/g)^2
    b = b.subs(g_f=g*sqrt(R_f))

    var('R_v',domain='real',  latex_name =r'R_{\nu}') #ratio (v/g)^2
    b = b.subs(v=g*sqrt(R_v))

    var('r_b',domain='real')
    b = b.subs(kappa_b=gamma/r_b)


    var('r_g',domain='real',latex_name =r'r_g')
    b = b.subs(gamma_g=gamma*r_g)


    var('r_f',domain='real',latex_name =r'r_f')
    b = b.subs(gamma_f=gamma*r_f)


    b = b.subs(g = sqrt(C*kappa_c*gamma)) 
    
    b = b.subs(r_b = c/(C*R_v))
    
    b = b.subs(dc0 = (dct+0.5*I)*kappa_c )
    
    
    L_elem = b
    
  
    L_elem = L_elem._mathematica_().PowerExpand()._sage_()
    L_elem = L_elem._mathematica_().Factor()._sage_()


    L_elem = MMA_simplify(L_elem,full=not True)

    L_elem = SR(str(L_elem).replace('Sqrt','sqrt'))

    

    show( symround(L_elem))

    print(f' |{s1.gs_e1_dec_states[L_nonzeros_pos[which][0]]}> < {s1.gs_e1_dec_states[L_nonzeros_pos[which][1]]} | \n')
    print(f' Original coefficient :')
    show(s1.L_coeffs[lind_op])
    return L_elem


## View Lindblad results

In [24]:
lind_op = 1  #which lindblad operator
which   = 1  #which non zero element to access
L_elem = lind(lind_op,which)

print(latex(symround(L_elem)))


(C + 0.50*I*Det)*Omega*sqrt(r_f)/((C*Det*R_f + C*DEt + 0.50*I*DEt*Det)*sqrt(gamma))

 |0f01> < 0g01 | 

 Original coefficient :


sqrt(gamma_f)

\frac{{\left(C + 0.50i \, {\tilde{\Delta }_e}\right)} {\Omega} \sqrt{{r_f}}}{{\left(C {\tilde{\Delta }_e} R_{f} + C {\tilde{\Delta }_E} + 0.50i \, {\tilde{\Delta }_E} {\tilde{\Delta }_e}\right)} \sqrt{{\gamma}}}


In [40]:
factor = -8


print('numerator')
nu = symround(SR(str(MMA_simplify(factor*(L_elem.numerator().expand()),full=not True)).replace('Sqrt','sqrt'))\
              ,digits=3)
show(nu)
print("\n\n Latex")
print(latex(nu))

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print('\n\ndenominator')
den = symround(SR(str(MMA_simplify(factor*(L_elem.denominator().expand()),full=not True)).replace('Sqrt','sqrt'))\
               ,digits=3)
show(den)
print("\n\n Latex")
print(latex(den))

numerator


(2*(16*c^2 + 12*c + 1)*C*(De0t + Det) + (48*I*c^2 + 16*I*c + I)*De0t*Det + C^2*(-32*I*c - 4*I))*Omega*sqrt(r_g)



 Latex
{\left(2 \, {\left(16 \, {c}^{2} + 12 \, {c} + 1\right)} C {\left({\tilde{\Delta }_{e0}} + {\tilde{\Delta }_e}\right)} + {\left(48 i \, {c}^{2} + 16 i \, {c} + i\right)} {\tilde{\Delta }_{e0}} {\tilde{\Delta }_e} + C^{2} {\left(-32 i \, {c} - 4 i\right)}\right)} {\Omega} \sqrt{{r_g}}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


denominator


((32*De0t*Det*(c + 0.250)^2 + (-16*I*De0t*c - 16*I*Det*c - 4*I*De0t - 4*I*Det)*C - 8*C^2)*C*R_f + (2*(16*c^2 + 12*c + 1)*C*(De0t + Det) + (48*I*c^2 + 16*I*c + I)*De0t*Det + C^2*(-32*I*c - 4*I))*DEt)*sqrt(gamma)



 Latex
{\left({\left(32 \, {\tilde{\Delta }_{e0}} {\tilde{\Delta }_e} {\left({c} + 0.250\right)}^{2} + {\left(-16 i \, {\tilde{\Delta }_{e0}} {c} - 16 i \, {\tilde{\Delta }_e} {c} - 4 i \, {\tilde{\Delta }_{e0}} - 4 i \, {\tilde{\Delta }_e}\right)} C - 8 \, C^{2}\right)} C R_{f} + {\left(2 \, {\left(16 \, {c}^{2} + 12 \, {c} + 1\right)} C {\left({\tilde{\Delta }_{e0}} + {\tilde{\Delta }_e}\right)} + {\left(48 i \, {c}^{2} + 16 i \, {c} + i\right)} {\tilde{\Delta }_{e0}} {\tilde{\Delta }_e} + C^{2} {\left(-32 i \, {c} - 4 i\right)}\right)} {\tilde{\Delta }_E}\right)} \sqrt{{\gamma}}


# Expand linearly in the limit:   $C\rightarrow \infty$ routine

In [ ]:
#with mathematica
q = b.subs(C=1/x)
q = MMA_simplify(q)
tayl = q._mathematica_().Series((x,0,1)).Normal()._sage_()
tayl = tayl.subs(x=1/C)

show(symround(tayl))


((2*I*R_v*c_k + 0.66*I)*R_v*c_k + 0.042*I)/(((2*I*DE + 1.3*R_f)*R_v^2*c_k^2 + (0.66*I*DE + R_f)*R_v*c_k + 0.042*I*DE + 0.083*R_f)*C)

# ~v~v~v~v~v~v~v~v~v~v~v~~v~v~v~v~v~v~v~v~vv

In [2]:
def substit(a):
    np.random.seed(31)
    
    a = a.subs(g0=g,gamma0=gamma)
    
    a = MMA_simplify((a.subs(db=np.random.random(),
            #De0=np.random.random(),
             #De=np.random.random(),
             gamma=np.random.random(),
             Omega=C^(1/2)*np.random.random(),
             g=C^(1/2)*np.random.random(),#np.random.random(),
             v=C^(1/2)*np.random.random(),#np.random.random(),
             g0=C^(1/2)*np.random.random(),#np.random.random(),
             g_f=C^(1/2)*np.random.random(),#np.random.random(),
             gamma_g = np.random.random(),
             gamma_f=np.random.random(),
             gamma0=np.random.random(),
             kappa_c=np.random.random(),
             #dc=np.random.random(),
             phi=np.random.random(),
             kappa_b=np.random.random())))
    a = symround(a,show_del=False,tol=1e-10)#.subs(DE=np.random.random()).n()

    return a

# Check if by flipping detunings we can achieve same losses

In [3]:
L_nonzeros = []
L_nonzeros_pos = []
for i in range(len(s1.lindblad_list)):
    print(i)
    L_matrix_orig = s1.lindblad_list[i]
    L_matrix      = s1.eff_lindblad_list[i] #lindblad_list[lind_op] ##(gamma_g=0,gamma_f=gamma)

    L_nonzeros.append([])
    L_nonzeros_pos.append([])
    #find non zero elements
    for i in  range(L_matrix.nrows()):
        for j in  range(L_matrix.ncols()):
            if str(L_matrix[i,j])!= '0.0' and str(L_matrix[i,j])!= '0': #not L_matrix[i,j].is_zero():
                L_nonzeros[-1].append(L_matrix[i,j])
                L_nonzeros_pos[-1].append((i,j))


0
1
2
3
4
5
6
7
8
9
10


# O-x-O

In [22]:
a = s1.eff_hamiltonian_gs.subs(dc=DE,db=DE,De=2*DE,De0=0)

b = s1.eff_hamiltonian_gs.subs(dc=-DE,db=-DE,De0=-1.5*DE,De=-0.5*DE,DE=-DE)

#show(substit(s1.eff_hamiltonian_gs[0,0]))
#q = substit(a+b)
q = substit(a).numpy().diagonal()+substit(b).numpy().diagonal()[::-1]
show(q)

array([0.49*(0.054*DE^6 + 0.10*DE^4 + (DE^2 + 0.91)*C^2 + (-0.47*DE^4 - 0.45*DE^2 - 0.021)*C + 0.046*DE^2 + 0.000092)*C*DE/(0.045*DE^8 + 0.083*DE^6 + 0.039*DE^4 + (DE^4 + 0.97*DE^2 + 0.047)*C^2 + (-0.42*DE^6 - 0.45*DE^4 - 0.050*DE^2 + 0.00027)*C + 0.00027*DE^2 + 3.9e-7) + 0.72*(-1.2*DE^2 + C - 0.0024)*C*DE/(1.5*DE^4 - 2.4*C*DE^2 + C^2 + 0.010*DE^2 + 0.0078*C + 0.000015),
       -0.22*(1.6*DE^4 + (-2.8*DE^2 - 0.058)*C + C^2 + 0.093*DE^2 + 0.00018)*C*DE/(0.58*DE^6 + 0.038*DE^4 + (DE^2 + 0.031)*C^2 + (-1.5*DE^4 - 0.049*DE^2 + 0.00021)*C + 0.00024*DE^2 + 3.5e-7) + 0.42*(0.19*DE^4 + (-0.88*DE^2 - 0.11)*C + C^2 + 0.17*DE^2 + 0.00034)*C*DE/(0.13*DE^6 + 0.12*DE^4 + (DE^2 + 0.11)*C^2 + (-0.72*DE^4 - 0.18*DE^2 + 0.00075)*C + 0.00086*DE^2 + 1.2e-6),
       -0.22*(1.6*DE^4 + (-2.8*DE^2 - 0.058)*C + C^2 + 0.093*DE^2 + 0.00018)*C*DE/(0.58*DE^6 + 0.038*DE^4 + (DE^2 + 0.031)*C^2 + (-1.5*DE^4 - 0.049*DE^2 + 0.00021)*C + 0.00024*DE^2 + 3.5e-7) + 0.42*(0.19*DE^4 + (-0.88*DE^2 - 0.11)*C + C^2 + 0.17*DE^2 + 0.00034)*C*DE/(0.13*DE^6 + 0.12*DE^4 + (DE^2 + 0.11)*C^2 + (-0.72*DE^4 - 0.18*DE^2 + 0.00075)*C + 0.00086*DE^2 + 1.2e-6),
       -0.32*(0.57*DE^6 + 0.066*DE^4 + (DE^2 + 0.057)*C^2 + (-1.6*DE^4 - 0.11*DE^2 - 0.00087)*C + 0.0020*DE^2 + 3.8e-6)*C*DE/(0.31*DE^8 + 0.038*DE^6 + 0.0013*DE^4 + (DE^4 + 0.079*DE^2 + 0.0013)*C^2 + (-1.1*DE^6 - 0.088*DE^4 - 0.0013*DE^2 + 7.3e-6)*C + (7.6e-6)*DE^2 + 1.0e-8) - 0.72*(-1.2*DE^2 + C - 0.0024)*C*DE/(1.5*DE^4 - 2.4*C*DE^2 + C^2 + 0.010*DE^2 + 0.0078*C + 0.000015)],
      dtype=object)

In [6]:
# gate time
va = substit(a).numpy().diagonal()

den = va[0]+va[3]-va[1]-va[2]

den = den._mathematica_().Together()._sage_()

tg = np.pi/den

In [7]:
for which in range(4):

    for lind_op in range(len(s1.lindblad_list)):
         #which non zero element to access
        print(lind_op,which,s1.L_coeffs[lind_op])
        if str(s1.L_coeffs[lind_op])=='sqrt(gamma)':
            try:   
                a = L_nonzeros[lind_op][which].subs(dc=DE,db=DE,De=2*DE,De0=0)
                for which2 in range(4):
                    b = L_nonzeros[lind_op+1][which2].subs(dc=-DE,db=-DE,De0=-2*DE,De=0,DE=-DE)



                    s = substit(a*conjugate( a)  - b * conjugate(b))
                    #s = symround(MMA_simplify(s.subs(De0=np.random.random(),De=np.random.random(),R0=np.random.random(),gamma=np.random.random(),Omega=np.random.random(),C=np.random.random(),c=np.random.random(),R_f=np.random.random(),r_g = np.random.random(),r0=np.random.random(),r_f=np.random.random(),kappa_c=np.random.random(),dc0=np.random.random(),DE=np.random.random(),gamma0=np.random.random(),R_v=np.random.random())._mathematica_().Together()._sage_().numerator()))


                    if s == 0 :
                        print('Yesssssssssssssssssss')
                    else:
                        print('no')
                        print(s)
            except:
                pass
        try:   
            a = L_nonzeros[lind_op][which].subs(dc=DE,db=DE,De=2*DE,De0=0)

            b = L_nonzeros[lind_op][3-which].subs(dc=-DE,db=-DE,De0=-2*DE,De=0,DE=-DE)



            s = substit(a*conjugate( a)  - b * conjugate(b))
            #s = symround(MMA_simplify(s.subs(De0=np.random.random(),De=np.random.random(),R0=np.random.random(),gamma=np.random.random(),Omega=np.random.random(),C=np.random.random(),c=np.random.random(),R_f=np.random.random(),r_g = np.random.random(),r0=np.random.random(),r_f=np.random.random(),kappa_c=np.random.random(),dc0=np.random.random(),DE=np.random.random(),gamma0=np.random.random(),R_v=np.random.random())._mathematica_().Together()._sage_().numerator()))

            
            if s == 0 :
                print('Yesssssssssssssssssss')
            else:
                print('no')
                print(s)
        except:
            pass



0 0 sqrt(kappa_c)
Yesssssssssssssssssss
1 0 sqrt(gamma)
no
((-(7.3e-6)*DE^6 - (3.4e-7)*DE^4 + (4.7e-8)*DE^2)*C^8 + (0.000049*DE^8 - (3.3e-6)*DE^6 - (9.5e-7)*DE^4 - (2.9e-9)*DE^2)*C^7 + (-0.00012*DE^10 + 0.000027*DE^8 + (4.1e-6)*DE^6 - (2.7e-8)*DE^4 - (3.3e-10)*DE^2)*C^6 + (0.00013*DE^12 - 0.000068*DE^10 - (8.6e-6)*DE^8 + (1.4e-8)*DE^6 - (1.7e-9)*DE^4)*C^5 + (-0.000053*DE^14 + 0.000072*DE^12 + (8.7e-6)*DE^10 + (1.5e-9)*DE^8 - (3.8e-9)*DE^6)*C^4 + (-0.000027*DE^13 - (3.0e-6)*DE^11 + (1.1e-7)*DE^9 + (1.2e-8)*DE^7)*C^3*DE)*C^5/((-3.9*DE^8 + 5.3*I*DE^7 + 2.9*DE^6 - 0.83*I*DE^5 + (DE^2 - 0.63*I*DE - 0.10)*C^3 - 0.13*DE^4 + (-4.9*DE^4 + 4.3*I*DE^3 + 1.3*DE^2 - 0.15*I*DE - 0.0045)*C^2 + 0.012*I*DE^3 + (7.8*DE^6 - 8.8*I*DE^5 - 3.8*DE^4 + 0.79*I*DE^3 + 0.080*DE^2 - 0.0036*I*DE - 0.000058)*C + 0.00060*DE^2 - 0.000015*I*DE - 1.6e-7)*(-3.9*DE^8 - 5.3*I*DE^7 + 2.9*DE^6 + 0.83*I*DE^5 + (DE^2 + 0.63*I*DE - 0.10)*C^3 - 0.13*DE^4 + (-4.9*DE^4 - 4.3*I*DE^3 + 1.3*DE^2 + 0.15*I*DE - 0.0045)*C^2 - 0.012*I*D

Yesssssssssssssssssss
5 2 sqrt(gamma_f)
Yesssssssssssssssssss
6 2 sqrt(gamma_g)
Yesssssssssssssssssss
7 2 sqrt(kappa_b)
Yesssssssssssssssssss
8 2 sqrt(kappa_c)
Yesssssssssssssssssss
9 2 sqrt(gamma)
10 2 sqrt(gamma0)
0 3 sqrt(kappa_c)
Yesssssssssssssssssss
1 3 sqrt(gamma)
2 3 sqrt(gamma0)
3 3 sqrt(kappa_b)
Yesssssssssssssssssss
4 3 sqrt(kappa_c)
Yesssssssssssssssssss
5 3 sqrt(gamma_f)
no
(-4.1*I*DE^6 + 2.7*I*DE^4 + (-1.5*I*DE^2 + 0.20*I)*C^2 + 0.093*I*C^3 + (4.8*I*DE^4 - 1.6*I*DE^2 + 0.013*I)*C - 0.090*I*DE^2 + 0.00019*I)*C*DE/((-3.9*DE^8 + 5.3*I*DE^7 + 2.9*DE^6 - 0.83*I*DE^5 + (DE^2 - 0.63*I*DE - 0.10)*C^3 - 0.13*DE^4 + (-4.9*DE^4 + 4.3*I*DE^3 + 1.3*DE^2 - 0.15*I*DE - 0.0045)*C^2 + 0.012*I*DE^3 + (7.8*DE^6 - 8.8*I*DE^5 - 3.8*DE^4 + 0.79*I*DE^3 + 0.080*DE^2 - 0.0036*I*DE - 0.000058)*C + 0.00060*DE^2 - 0.000015*I*DE - 1.6e-7)*(-3.9*DE^8 - 5.3*I*DE^7 + 2.9*DE^6 + 0.83*I*DE^5 + (DE^2 + 0.63*I*DE - 0.10)*C^3 - 0.13*DE^4 + (-4.9*DE^4 - 4.3*I*DE^3 + 1.3*DE^2 + 0.15*I*DE - 0.0045)*C^2 - 0.012*

# Check scalings

In [8]:
def obtain_exponent(expr):
    expr = SR(str(( expr)).replace('Sqrt','sqrt'))
    try:
        exp = expr.op[0].op[0].op[0].op[1]
    except:
        try:
            exp = expr.op[0].op[0].op[1]
        except:
            try:
                exp = expr.op[0].op[1]
            except:
                exp = 0
    return exp

def obtain_expr_exp(expr):
    try:
        el = 0    
        q = (expr._mathematica_().Numerator()._sage_() .expand().operands()[el])
        q = SR(str(( q)).replace('Sqrt','sqrt'))
        test_bool = (str(q._mathematica_().Numerator()._sage_().expand().denominator()) == str(1))
        num_exp = (obtain_exponent(q._mathematica_().Numerator()._sage_().expand()))
        den1_exp = (obtain_exponent(q._mathematica_().Denominator()._sage_().expand()))
        den2_exp = (obtain_exponent(expr._mathematica_().Denominator()._sage_().expand()))
        res = num_exp-den1_exp-den2_exp
        if not test_bool:
            #show(q)
            res = obtain_exponent(q.numerator())-obtain_exponent(q.denominator())
            return res
        return res
    except:
        #print('altern')
        den_exp = obtain_exponent( expr._mathematica_().Denominator()._sage_().expand())
        num_exp = obtain_exponent( expr._mathematica_().Numerator()._sage_().expand())
        return num_exp-den_exp

In [11]:
n =1/2

dbool = 1
# gate time

a = s1.eff_hamiltonian_gs.subs(dc=dbool*DE,db=dbool*DE,De=1.5*DE,De0=0.5*DE)

a = a.subs(DE=C^n)

va = substit(a).numpy().diagonal()


den = va[0]+va[3]-va[1]-va[2]

den = den._mathematica_().Together()._sage_()

#den = den.numerator().expand() / den.denominator().expand()

tg_sc = - obtain_expr_exp(den)

print(tg_sc)

-0.5


In [12]:
for which in range(4):

    for lind_op in range(len(s1.lindblad_list)):
         #which non zero element to access
        #print(lind_op,which,s1.L_coeffs[lind_op])

        try:   
            a = L_nonzeros[lind_op][which].subs(dc=dbool*DE,db=dbool*DE,De=1.5*DE,De0=0*DE)#subs(dc=DE,db=DE,De=3/2*DE,De0=0.5*DE)
            a = a.subs(DE=C^n)
            loss_rate = substit(a*conjugate( a))
            
            loss_rate_sc = obtain_expr_exp(loss_rate)
            
            loss_sc =  tg_sc + loss_rate_sc
            
            print(loss_sc)

            if loss_sc>=0:
                print(s1.L_coeffs[lind_op], which)
                show(loss_rate)
                
        except:
            pass



-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5
-0.5


In [69]:
show(tg)

(DE^42 - 5.0*DE^40 + 9.839590999999999*DE^38 - 9.153792600000001*DE^36 + 3.3430946500000003*DE^34 + 0.42843207*DE^32 - 0.4185541695300001*DE^30 + 12.487*DE^28*e - 12.071423908885*DE^28 - 37.071149000000005*DE^26*e + 33.23688380478215*DE^26 + 31.854119064*DE^24*e - 22.521730131019012*DE^24 + 1.0642545255999978*DE^22*e - 10.368512546906494*DE^22 - 10.032484587*DE^20*e + 13.22237633132717*DE^20 + 0.36684672055000106*DE^18*e + 0.10376123011194059*DE^18 + 1.0167461001240001*DE^16*e - 1.2310855136476688*DE^16 + 30.364131999999998*DE^14*e^2 - 74.64764741199741*DE^14*e + 35.6458729847336*DE^14 - 35.444680796*DE^12*e^2 + 74.97705845025796*DE^12*e - 19.365077204770603*DE^12 - 10.383373028399998*DE^10*e^2 + 46.953156949087024*DE^10*e - 51.001446597730116*DE^10 + 13.545375109999998*DE^8*e^2 - 40.56454207930957*DE^8*e + 29.013719395745*DE^8 + 2.7687990764399992*DE^6*e^2 - 8.592708723295434*DE^6*e + 6.476494568492593*DE^6 + 0.28315251408799996*DE^4*e^2 - 0.8974175914675755*DE^4*e + 0.6962031589900969*DE^4 + 0.014075892070439997*DE^2*e^2 - 0.04505120276014457*DE^2*e + 0.0354061057923555*DE^2 + 0.00036060471611999993*e^2 - 0.00116742786*e + 0.00093114)/(0.01552560000000014*DE^39 - 0.031725235599999735*DE^37 + 0.010112121359999282*DE^35 + 0.011035977940000247*DE^33 - 0.002502129039999912*DE^31 + 1.907*DE^29*e + 5.997857238469798*DE^29 + 0.351954800000005*DE^27*e - 20.826694120585696*DE^27 - 8.607796079200007*DE^25*e + 24.60461596137386*DE^25 + 7.168159385400005*DE^23*e - 9.049962577942402*DE^23 + 0.6833078511799968*DE^21*e - 2.255955882485777*DE^21 - 1.1833374117480007*DE^19*e + 1.2322028501167055*DE^19 - 0.2732855788169999*DE^17*e + 0.27772614639150905*DE^17 - 30.784622000000006*DE^15*e^2 + 78.83307573678873*DE^15*e - 35.985118181178606*DE^15 + 6.864249954800009*DE^13*e^2 - 8.54476418999738*DE^13*e - 15.85195318778755*DE^13 + 17.134245014599987*DE^11*e^2 - 51.093199880219906*DE^11*e + 37.81993290293532*DE^11 + 3.092774958999998*DE^9*e^2 - 9.308184086554856*DE^9*e + 8.108960904256094*DE^9 - 0.2208609679800002*DE^7*e^2 + 0.8506878436514711*DE^7*e - 0.5792426270159492*DE^7 - 0.0392315267263999*DE^5*e^2 + 0.14252522354765673*DE^5*e - 0.10824732825625916*DE^5 - 0.002335077388540002*DE^3*e^2 + 0.008495664495473447*DE^3*e - 0.006745324435199311*DE^3 + (2.7184169999988454e-07)*DE*e^2 + (2.3064339999999883e-05)*DE*e - (1.9739999999999862e-05)*DE)